# Predicting Newsletter Subscription in Minecraft Players

## Introduction:

### Background
The studying of player engagement and behavior in games such as Minecraft is a growing in the field of data science. In this project, I will be wokring with a research group at the University of British Columbia led by Dr. Frank Wood, which is investigating the behaviours and activities of players in a custom Minecraft game server. They collected each player’s activity over time, which allowed them to study their behavior, preferences, and play patterns.

### Question (I Wrote)
The aim of this project is to answer the following question:
"Can characteristics of a player (e.g., behavior, play frequency, time-of-day usage) predict whether they will subscribe to the game-related newsletter?"
This question is quite intersting as it can help identify the types of players who most likley to show more engaggemnet and interests in the projects. This will also help with gettings player who have more long-term interest, which will help prioritize players for future studies.

### Data Description
To answer this question, this project will use two datasets:

* `players.csv`: Contains demographic and behavioral features of individual players, such as age, device, and whether they are subscribed to a newsletter.
* `sessions.csv`: Contains a record of each game session for each player, whcih includes timestamps, session length, and actions taken.
In this project, these datasets will be wrangled and used to construct meaningful visuals and relationships related to player behavior and characteristics to answer the aim/qu. Then, we will use exploratory data analysis and machine learning models to identify which features are most predictive of newsletter subscription.



## Methods and Results:

### Reading and Loading the Data
First let's load the libary `tidyverse` to get a package of functions. Then, lets read both of the datasets using the function `read_csv` and assign them to an object.